In [1]:
import requests
import pandas as pd
import numpy as np
import time 
import json
import os
from datetime import datetime, timedelta



# REQUEST AND CLEAN API 

In [ ]:
# API requests: 
btc = requests.get("https://api.cryptowat.ch/markets/kraken/btcusd/ohlc?periods=86400&apikey=XVGSK3GTQSR4WA9W8XY7")

In [ ]:
#Jsonize
btc = btc.json()

In [ ]:
btc['result']['86400']

In [ ]:
# Convert jsonized lists to DF 
btc = pd.DataFrame(btc['result']['86400'])

In [ ]:
# Rename DF columns 
btc.columns = ['date', 'open_price', 'hi_price', 'lo_price', 'close_price', 'vol', 'quote_vol']

In [ ]:
# Select only date and closing value 
btc = btc[['date','close_price']]

In [ ]:
# Convert Epoch time (secs since 1970) into datetime object
btc['date'] = pd.to_datetime(btc['date'], unit='s')

In [ ]:
#Select date range from 2016 - 2021
BTC = (btc.loc[btc["date"].between("2016-01-01", "2021-12-31")]).reset_index(drop=True)

# CLEAN API DATA FUNCTION

In [ ]:
def clean_api_request(req): #Inouts a cryptowatch API request
    req = req.json()
    # Convert jsonized lists to DF 
    req = pd.DataFrame(req['result']['86400'])
    # Rename DF columns 
    req.columns = ['date', 'open_price', 'hi_price', 'lo_price', 'close_price', 'vol', 'quote_vol']
    # Select only date and closing value 
    req = req[['date','close_price']]
    # Convert Epoch time (secs since 1970) into datetime object
    req['date'] = pd.to_datetime(req['date'], unit='s')
    #Select date range from 2016 - 2021
    req = (req.loc[req["date"].between("2016-01-01", "2021-12-31")]).reset_index(drop=True)
    return req # returns a DF with date and daily close cryptocurrency price for every request

# COMODITIES 

In [184]:
import pandas as pd 
OIL_BRENT = pd.read_csv("../data/brent-crude-oil-prices-16-21.csv")
OIL_WTI = pd.read_csv("../data/WTI_oil_prices_16-21.csv")
GOLD = pd.read_csv("../data/gold_prices_16-21.csv")
SILVER = pd.read_csv("../data/silver_prices_16-21.csv")
STEEL = pd.read_csv("../data/Steel_prices_16-21.csv")
# COAL = pd.read_csv("../data/Coal_p_16-21.csv")

In [192]:
com_list = [OIL_WTI, OIL_BRENT, GOLD, SILVER,STEEL]  # List of dataframes

In [193]:
# We convert the date coliumn into the index to compare later with other DF 
OIL_WTI = OIL_WTI.set_index('date')
OIL_BRENT = OIL_BRENT.set_index('date')
GOLD = GOLD.set_index('date')
SILVER = SILVER.set_index('date')
STEEL = STEEL.set_index('date')

In [197]:
# Merging DF
merge=pd.merge(OIL_WTI,OIL_BRENT, how='inner', left_index=True, right_index=True)
merge_2 = pd.merge(merge,GOLD, how='inner', left_index=True, right_index=True)
merge_3 = pd.merge(merge_2,SILVER, how='inner', left_index=True, right_index=True)
comm_df = pd.merge(merge_3,STEEL, how='inner', left_index=True, right_index=True)

# CURRENCIES (£-USD, EURO-USD, USD-Yuan, USD-yen)

In [209]:
import pandas as pd 
gbp = pd.read_csv("../data/USD_GBP_16-21.csv")
eur = pd.read_csv("../data/USD_EUR_16-21.csv")
yua = pd.read_csv("../data/USD_CNY_16-21.csv")
yen = pd.read_csv("../data/USD_JPY_16-21.csv")

In [210]:
gbp['date'] = pd.to_datetime(gbp['date'])
gbp['date'] = gbp['date'].dt.strftime('%d/%m/%Y')
eur['date'] = pd.to_datetime(eur['date'])
eur['date'] = eur['date'].dt.strftime('%d/%m/%Y')
yua['date'] = pd.to_datetime(yua['date'])
yua['date'] = yua['date'].dt.strftime('%d/%m/%Y')
yen['date'] = pd.to_datetime(yen['date'])
yen['date'] = yen['date'].dt.strftime('%d/%m/%Y')

In [211]:
gbp = gbp.set_index('date')
eur = eur.set_index('date')
yua = yua.set_index('date')
yen = yen.set_index('date')

In [212]:
currencies = [gbp,eur,yua,yen]

In [213]:
curr_df = pd.concat(currencies,axis=1,join="inner",ignore_index=False,sort=False, copy=False) 

In [313]:
curr_df

,usd_gbp,usd_gbp_ %,usd_eur,usd_eur_%,usd_yuan,usd_yuan_%,usd_yen,usd_yen_ %
date,,,,,,,,
01/01/2016,0.6781,-0.04%,0.9208,0.01%,6.4936,0.00%,120.33,0.01%
04/01/2016,0.6794,0.19%,0.9232,0.26%,6.5354,0.64%,119.44,-0.74%
05/01/2016,0.6815,0.31%,0.9304,0.78%,6.5148,-0.32%,119.06,-0.32%
06/01/2016,0.6836,0.31%,0.9275,-0.31%,6.5566,0.64%,118.47,-0.50%
07/01/2016,0.6841,0.07%,0.9144,-1.41%,6.5929,0.55%,117.68,-0.67%
...,...,...,...,...,...,...,...,...
27/12/2021,0.7438,-0.42%,0.8826,-0.08%,6.3705,0.05%,114.83,0.37%
28/12/2021,0.7442,0.05%,0.8839,0.15%,6.3684,-0.03%,114.81,-0.02%
29/12/2021,0.7412,-0.40%,0.8809,-0.34%,6.3680,-0.01%,114.94,0.11%


# CURRENCIES AND COMM concat 

In order to have the same rows and dates to compare

In [217]:
main_df = pd.merge(curr_df,comm_df,how="inner",on='date')

# INTEREST RATE INDEXES 

In [250]:
import pandas as pd 
libor = pd.read_csv("../data/LIBOR_16-21.csv")
eonia = pd.read_csv("../data/eonia_16-21.csv")
fed = pd.read_csv("../data/fed-funds-rate-16-21.csv")
# Create a list of indexes to concat, ast hey have the smae days of cotization)
interest_list = [libor, eonia,fed]

In [251]:
# convert date columun to datetime object and set it as index 
libor['date'] = pd.to_datetime(libor['date'])
libor['date'] = libor['date'].dt.strftime('%d/%m/%Y')
libor = libor.set_index('date')

eonia['date'] = pd.to_datetime(eonia['date'])
eonia['date'] = eonia['date'].dt.strftime('%d/%m/%Y')
eonia = eonia.set_index('date')

fed['date'] = pd.to_datetime(fed['date'])
fed['date'] = fed['date'].dt.strftime('%d/%m/%Y')
fed = fed.set_index('date')

In [253]:
rates_1 =  pd.merge(libor,eonia,how="inner",on='date')

In [254]:
rates =  pd.merge(rates_1,fed,how="inner",on='date')

# Merge rates with main DF 

In [257]:
main_=  pd.merge(main_df,rates,how="inner",on='date')

# Merge cryptos with main DF

In [318]:
BTC = pd.read_csv("../final_data/btc.csv")
LTC = pd.read_csv("../final_data/ltc.csv")
ETH = pd.read_csv("../final_data/eth.csv")
XRP = pd.read_csv("../final_data/xrp.csv")
DOGE = pd.read_csv("../final_data/doge.csv")

,close_price
date,
01/01/2016,431.31000
02/01/2016,431.50530
03/01/2016,434.98000
04/01/2016,426.57000
05/01/2016,431.08414
...,...
27/12/2021,50797.30000
28/12/2021,50687.40000
29/12/2021,47548.10000


In [319]:
# Convert date column to desired data type:
XRP["Date"] = pd.to_datetime(XRP["Date"],infer_datetime_format=True)
XRP['Date'] = XRP['Date'].dt.normalize()

DOGE["Date"] = pd.to_datetime(DOGE["Date"],infer_datetime_format=True)
DOGE['Date'] = DOGE['Date'].dt.normalize()

# Drop 'Unnamed:0' column: 
BTC = BTC.drop('Unnamed: 0', axis=1)
LTC = LTC.drop('Unnamed: 0', axis=1)
ETH = ETH.drop('Unnamed: 0', axis=1)
XRP = XRP.drop('Unnamed: 0', axis=1)
DOGE = DOGE.drop('Unnamed: 0', axis=1)


In [320]:
# Convert date column to desired format:
BTC['date'] = pd.to_datetime(BTC['date'])
BTC['date'] = BTC['date'].dt.strftime('%d/%m/%Y')
LTC['date'] = pd.to_datetime(LTC['date'])
LTC['date'] = LTC['date'].dt.strftime('%d/%m/%Y')
ETH['date'] = pd.to_datetime(ETH['date'])
ETH['date'] = ETH['date'].dt.strftime('%d/%m/%Y')
XRP['~Date'] = pd.to_datetime(XRP['Date'])
XRP['Date'] = XRP['Date'].dt.strftime('%d/%m/%Y')
DOGE['Date'] = pd.to_datetime(DOGE['Date'])
DOGE['Date'] = DOGE['Date'].dt.strftime('%d/%m/%Y')
# Set date column as index to merge and have the same days of trade: 
BTC = BTC.set_index('date')
LTC = LTC.set_index('date')
ETH = ETH.set_index('date')
XRP = XRP.set_index('Date')
DOGE= DOGE.set_index('Date')

In [321]:
crypto_list = [BTC,LTC,ETH, XRP,DOGE]

In [322]:
cryptos = pd.concat(crypto_list,axis=1,join="inner",ignore_index=False,sort=False, copy=False)



In [323]:
cryptos 

,close_price,close_price,close_price,xrp_p,~Date,doge_p
01/01/2016,431.31000,3.48010,0.94885,0.005955,2016-01-01,0.000154
02/01/2016,431.50530,3.48500,0.92037,0.006542,2016-01-02,0.000270
03/01/2016,434.98000,3.50000,0.95000,0.008006,2016-01-03,0.000249
04/01/2016,426.57000,3.49300,0.96000,0.007418,2016-01-04,0.000224
05/01/2016,431.08414,3.43589,0.00000,0.006791,2016-01-05,0.000226
...,...,...,...,...,...,...
27/12/2021,50797.30000,156.00000,4064.33000,0.926184,2021-12-27,0.926184
28/12/2021,50687.40000,155.84000,4038.76000,0.852960,2021-12-28,0.852960
29/12/2021,47548.10000,145.64000,3792.09000,0.816714,2021-12-29,0.816714
30/12/2021,46473.20000,145.48000,3628.29000,0.839649,2021-12-30,0.839649
